# Working with MEG dataset
MEG dataset from THINGS initiative: https://openneuro.org/datasets/ds004212/versions/2.0.1
### Training data partition
The data is .fif file which needs to be converted into a rather easy to use .npy array.
We will use mne library for this: https://mne.tools/dev/index.html 


We collected extensively sampled object representations using magnetoencephalography (MEG). To this end, we drew on the THINGS database (Hebart et al., 2019), a richly-annotated database of 1,854 object concepts representative of the American English language which contains 26,107 manually-curated naturalistic object images.

ERP = event-related potential; signal with respect to some event
VEP = ERP for visual stimuli



27084 events, 281 time points, 271 channels

In [1]:
import numpy as np
import mne, os
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import matplotlib.image as mpimg
import nimfa

In [3]:
channel_picks               = ['O','T','P']
file_path = 'preprocessed'
def load_epochs(file_path,all_epochs = []):
    epochs = mne.read_epochs(file_path, preload=False)
    return epochs
epochs = load_epochs(file_path)

/tmp/ipykernel_2039442/1096225509.py:4: RuntimeWarning: This filename (preprocessed) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(file_path, preload=False)


FileNotFoundError: File does not exist: "/home/asuryawanshi/Documents/Neural-Representations-using-Things-Database/MEG/notebooks/preprocessed"

In [ ]:
epochs.metadata

In [ ]:
name = channel_picks[0]
print(name, channel_picks)
# ctf_layout = mne.find_layout(epochs.info)
picks_epochs = [epochs.ch_names[i] for i in np.where([s[2]==name for s in epochs.ch_names])[0]]
ep1 = epochs[epochs.metadata['trial_type']=='exp']  
ep1.load_data() 
ep1.pick_channels(ch_names=picks_epochs); # supress output


In [ ]:
NOS_CONCEPTS = 1854
NOS_IMAGE_PER_CONCEPT = 12
NOS_CHANNELS_OPT = 39
NOS_TIME_POINTS = 281
time_points = epochs.times 

In [ ]:
# Load concept epochs if they already exist
# if os.path.exists('concept_epochs.npy'):
#     concept_epochs = np.load('concept_epochs.npy')
# else:
# Initialize the concept_epochs array with zeros
concept_epochs = np.zeros((NOS_IMAGE_PER_CONCEPT, NOS_CONCEPTS, NOS_TIME_POINTS, NOS_CHANNELS_OPT))

# Extract all data matching the condition in one go
indices = ep1.metadata['category_nr'].values - 1  # Adjust index (assuming category_nr starts at 1)
concept_epochs[:, indices, :, :] = ep1._data.transpose(0, 2, 1)
    
    # # Save the array
    # np.save('concept_epochs.npy', concept_epochs)

In [ ]:
concept_epochs.shape

In [ ]:
filtered_indices = (epochs._raw_times > 0.07) & (epochs._raw_times < 0.370)
concept_epochs = concept_epochs[:, :, filtered_indices, :]

In [ ]:
average_concept_epochs = concept_epochs.mean(axis = 0)
average_concept_epochs.shape

In [ ]:
z_scored_epochs = np.zeros_like(average_concept_epochs)
for channel in range(NOS_CHANNELS_OPT):
    mean = average_concept_epochs[:,channel,:].mean()
    stdev = average_concept_epochs[:,channel,:].std()
    z_scored_epochs[:,channel,:] = (average_concept_epochs[:,channel,:] - mean) / stdev
z_scored_epochs+= abs(z_scored_epochs.min()) # make all values positive  

In [ ]:

plt.hist(z_scored_epochs[:,2,:].flatten(), bins=100)
plt.title('Histogram of z-scored epochs for a sample channel')
plt.xlabel('amplitude')
plt.ylabel('# of occurrences') 
plt.show()
z_scored_epochs.shape 

In [ ]:
X = z_scored_epochs.reshape(NOS_CONCEPTS, average_concept_epochs.shape[1]*NOS_CHANNELS_OPT)
X.shape

In [ ]:
def fit_bnmf(V: np.ndarray, k: int):
    bnmf = nimfa.Bd(V, seed='random_c', rank=k, max_iter=500, min_residuals=1e-4, alpha=np.zeros((V.shape[0], k)),
                        beta=np.zeros((k, V.shape[1])), theta=.0, k=.0, sigma=1., skip=100, stride=1,
                        n_w=np.zeros((k, 1)), n_h=np.zeros((k, 1)), n_run=1, n_sigma=False)
    bnmf_fit = bnmf()
    W = bnmf_fit.basis()
    H = bnmf_fit.coef()
    return np.array(W), np.array(H)

def compute_evar_all(V: np.ndarray, W: np.ndarray, H: np.ndarray):
    V_hat = np.dot(W, H)
    rss = np.sum(np.asarray(V_hat - V)**2)
    evar_all = 1. - rss / (V*V).sum()
    return evar_all

def compute_evar_indiv(V: np.ndarray, W: np.ndarray, H: np.ndarray, d: int):
    V_hat_d = np.outer(W[:, d], H[d, :])
    rss = np.sum(np.asarray(V_hat_d - V)**2)
    evar_indiv = 1. - rss / (V*V).sum()
    return evar_indiv

def compute_evar_unique(V: np.ndarray, W: np.ndarray, H: np.ndarray, d: int, evar_all: float):
    V_hat_wo_d = np.dot(W[:, np.arange(W.shape[1]) != d], H[np.arange(H.shape[0]) != d, :])
    rss = np.sum(np.asarray(V_hat_wo_d - V)**2)
    evar_rest = 1. - rss / (V*V).sum()
    evar_unique = evar_all - evar_rest
    return evar_unique

$$X = W \times H$$
1854 is the number of data points and time*channels is the dimensionality of each data point.
$W$ is the matrix of basis vectors and $H$ is the coefficient of activity of this basis vectors. 

In [ ]:
import scipy.stats as st

def compute_log_likelihood(V: np.ndarray, W: np.ndarray, H: np.ndarray):
    """Compute log-likelihood."""
    V_hat = np.dot(W, H)
    err_std_dev = np.sqrt(np.var(V - V_hat))
    log_likelihood = st.norm.logpdf(V, loc=V_hat, scale=err_std_dev).sum()
    return log_likelihood

def compute_aic(V: np.ndarray, W: np.ndarray, H: np.ndarray):
    """Compute AIC."""
    log_likelihood = compute_log_likelihood(V, W, H)
    n_free_params = np.count_nonzero(W) + np.count_nonzero(H) + 1
    aic = 2 * n_free_params - 2 * log_likelihood
    return aic, n_free_params

def compute_bic(V: np.ndarray, W: np.ndarray, H: np.ndarray):
    """Compute BIC."""
    log_likelihood = compute_log_likelihood(V, W, H)
    I, J = V.shape
    n_samples = I * J
    n_free_params = np.count_nonzero(W) + np.count_nonzero(H) + 1
    bic = np.log(n_samples) * n_free_params - 2 * log_likelihood
    return bic

In [ ]:
nmf_components = 250 # 3 mins for 30 components 
BAYESIAN = 1

# Directory to save/load the matrices
directory = 'NMF_matrices'  

# Check if W, H exists in folder and load them
if os.path.isfile(f'{directory}/W_{nmf_components}.npy') and os.path.isfile(f'{directory}/H_{nmf_components}.npy'):
    W = np.load(f'{directory}/W_{nmf_components}.npy')
    H = np.load(f'{directory}/H_{nmf_components}.npy')
else:
    if BAYESIAN:
        W, H = fit_bnmf(X, nmf_components)
        X_pred = np.dot(W, H)
    else:
        model = NMF(n_components=nmf_components)
        W = model.fit_transform(X)
        H = model.components_
        X_pred = np.dot(W, H)
    np.save(f'{directory}/W_{nmf_components}.npy', W)
    np.save(f'{directory}/H_{nmf_components}.npy', H)

In [ ]:
print(compute_evar_all(X, W, H))

In [ ]:
array_dim = [2, 5, 10, 15, 20, 50, 100, 200, 250, 300]
W_dict = {} 
H_dict = {} 

for dim in array_dim:
    if os.path.isfile(f'{directory}/W_{dim}.npy') and os.path.isfile(f'{directory}/H_{dim}.npy'):
        W_dict[dim] = np.load(f'{directory}/W_{dim}.npy')
        H_dict[dim] = np.load(f'{directory}/H_{dim}.npy')
# # load matrices from directory
# W_2 = np.load(f'{directory}/W_2.npy')
# H_2 = np.load(f'{directory}/H_2.npy')
# W_5 = np.load(f'{directory}/W_5.npy')   
# H_5 = np.load(f'{directory}/H_5.npy')
# W_10 = np.load(f'{directory}/W_10.npy')
# H_10 = np.load(f'{directory}/H_10.npy')
# W_15 = np.load(f'{directory}/W_15.npy')
# H_15 = np.load(f'{directory}/H_15.npy')
# W_20 = np.load(f'{directory}/W_20.npy')
# H_20 = np.load(f'{directory}/H_20.npy')
# W_50 = np.load(f'{directory}/W_50.npy')
# H_50 = np.load(f'{directory}/H_50.npy')     
# W_100 = np.load(f'{directory}/W_100.npy')
# H_100 = np.load(f'{directory}/H_100.npy')   
# W_200 = np.load(f'{directory}/W_200.npy')                       
# H_200 = np.load(f'{directory}/H_200.npy')
# W_250 = np.load(f'{directory}/W_250.npy')
# H_250 = np.load(f'{directory}/H_250.npy')
# W_300 = np.load(f'{directory}/W_300.npy')
# H_300 = np.load(f'{directory}/H_300.npy')

BIC_array = np.zeros(len(array_dim))
AIC_array = np.zeros(len(array_dim))
explained_variance = np.zeros(len(array_dim))   
for i, dim in enumerate(array_dim):
    W = W_dict[dim]
    H = H_dict[dim]
    BIC_array[i] = compute_bic(X, W, H)
    AIC_array[i] = compute_aic(X, W, H)[0]
    explained_variance[i] = compute_evar_all(X, W, H)


In [ ]:
# plot AIC and BIC
plt.plot(array_dim, AIC_array, label='AIC')     
plt.plot(array_dim, BIC_array, label='BIC')    
plt.plot(array_dim, explained_variance, label='Explained Variance')         
plt.xlabel('Number of components')          
plt.ylabel('Information Criterion')                 
plt.legend()        
plt.show()  


In [ ]:
plt.plot(array_dim, explained_variance, label='Explained Variance')   
plt.xlabel('Number of components')
plt.ylabel('Explained Variance')            
plt.legend()    
plt.show()  

In [ ]:
array_dim = [2, 5, 10, 15, 20, 50, 100, 200, 250, 300]

# compute BIC 
print(f"BIC of 2 is {compute_bic(X, W_2, H_2)}")    
print(f"BIC of 5 is {compute_bic(X, W_5, H_5)}")
print(f"BIC of 10 is {compute_bic(X, W_10, H_10)}")  
print(f"BIC of 15 is {compute_bic(X, W_15, H_15)}")               
print(f"BIC of 20 is {compute_bic(X, W_20, H_20)}")             
print(f"BIC of 50 is {compute_bic(X, W_50, H_50)}")
print(f"BIC of 100 is {compute_bic(X, W_100, H_100)}")
print(f"BIC of 200 is {compute_bic(X, W_200, H_200)}")
print(f"BIC of 250 is {compute_bic(X, W_250, H_250)}")
print(f"BIC of 300 is {compute_bic(X, W_300, H_300)}")


In [ ]:
#plot bic's 
bic = [compute_bic(X, W_2, H_2), compute_bic(X, W_5, H_5), compute_bic(X, W_10, H_10), compute_bic(X, W_15, H_15), compute_bic(X, W_20, H_20), compute_bic(X, W_50, H_50), compute_bic(X, W_100, H_100), compute_bic(X, W_200, H_200), compute_bic(X, W_250, H_250), compute_bic(X, W_300, H_300)]
plt.plot([2, 5, 10, 15, 20, 50, 100, 200,250, 300], bic)
plt.xlabel('Number of components')
plt.ylabel('BIC')
plt.title('BIC vs number of components')
plt.show()

In [ ]:
W.shape, H.shape

In [ ]:
# indices which has max value on W
sorted_indices = np.argsort(W[:,0])
sorted_indices.shape

In [ ]:
nmf_components_vs_category = np.zeros((nmf_components, 10))
for i in range(nmf_components):
    sorted_indices = np.argsort(W[:,i])
    print(f'Categories which are best on component {i} are: {sorted_indices[-10:]}')
    nmf_components_vs_category[i,:] = sorted_indices[-10:]


In [ ]:
for component in range(nmf_components):
    print(f"Component {component} is best loaded by the following image categories:")
    for i in range(10):
        category_nr = nmf_components_vs_category[component, i] 
        # Get the image paths for the given category_nr
        image_paths = epochs.metadata[(epochs.metadata['category_nr'] == category_nr+1) & (epochs.metadata['trial_type'] == 'exp')]['image_path']
        
        if not image_paths.empty:
            # Print the folder name only for the first occurrence
            first_image_path = image_paths.iloc[0]
            folder_name = os.path.dirname(first_image_path)
            print(f"Category {category_nr} - {folder_name}")
        else:
            print(f"Category {category_nr} - No image paths found ")
            break

    print('\n')

In [ ]:
# plot images for the top 5 categories for each component   
for component in range(nmf_components):
    if component > 20:
        break
    print(f"Component {component} is best loaded by the following image categories:")
    fig, axs = plt.subplots(1, 10, figsize=(20, 20))
    for i in range(10):
        category_nr = nmf_components_vs_category[component, i] 
        # Get the image paths for the given category_nr
        image_paths = epochs.metadata[(epochs.metadata['category_nr'] == category_nr) & (epochs.metadata['trial_type'] == 'exp')]['image_path']
        image_path = image_paths.iloc[0]
        img = mpimg.imread(image_path)
        axs[i].imshow(img)
        axs[i].set_title(f"Category {category_nr}")
        axs[i].axis('off')
    plt.show()
    